In [1]:
## Load Dependencies
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

#import json, os, re, shutil, sys, time
import os, shutil, time
#from importlib import reload
from imp import reload
#import collections, itertools
import unittest
#from trainer import unittest
#from . import unittest
#import trainer.unittest as unittest
#from IPython.display import display, HTML

# NLTK for NLP utils and corpora
#import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from trainer import utils#, vocabulary, tf_embed_viz
#import utils
#import trainer.utils as utils

# rnnlm code
from trainer import rnnlm
#import trainer.rnnlm as rnnlm
#import rnnlm
reload(rnnlm)
#from trainer import rnnlm_test
#import trainer.rnnlm_test as rnnlm_test
#reload(rnnlm_test)
#from . import rnnlm; reload(rnnlm)
#from . import rnnlm_test; reload(rnnlm_test)
#import rnnlm; reload(rnnlm)
#import rnnlm_test; reload(rnnlm_test)
# packages for extracting data
import pandas as pd

#
#import cloudstorage as gcs
import random
import csv

In [ ]:
def make_tensorboard(tf_graphdir="/tmp/artificial_hotel_reviews/a4_graph", V=100, H=1024, num_layers=2):
    reload(rnnlm)
    TF_GRAPHDIR = tf_graphdir
    # Clear old log directory.
    shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)
    
    lm = rnnlm.RNNLM(V=V, H=H, num_layers=num_layers)
    lm.BuildCoreGraph()
    lm.BuildTrainGraph()
    lm.BuildSamplerGraph()
    summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)
    return summary_writer

# Unit Tests
def test_graph():
    reload(rnnlm); reload(rnnlm_test)
    utils.run_tests(rnnlm_test, ["TestRNNLMCore", "TestRNNLMTrain", "TestRNNLMSampler"])

def test_training():
    reload(rnnlm); reload(rnnlm_test)
    th = rnnlm_test.RunEpochTester("test_toy_model")
    th.setUp(); th.injectCode(run_epoch, score_dataset)
    unittest.TextTestRunner(verbosity=2).run(th)

In [ ]:
def score_each_step(lm, session, ids):
    #no batching
    bi = utils.rnnlm_batch_generator(ids, batch_size=100, max_time=100)
    for i, (w,y) in enumerate(bi):
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})
        feed_dict = {lm.input_w_:w,
                     lm.target_y_:y,
                     lm.learning_rate_: 0.002,
                     lm.use_dropout_: False,
                     lm.initial_h_:h}
        cost, h, _ = session.run([loss, lm.final_h_, train_op],feed_dict=feed_dict)
        #pick up here

In [30]:
## Training Functions
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=5, learning_rate=None):
    assert(learning_rate is not None)
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step0_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation

    for i, (w, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})

        #### YOUR CODE HERE ####
        feed_dict = {lm.input_w_:w,
                     lm.target_y_:y,
                     lm.learning_rate_: learning_rate,
                     lm.use_dropout_: use_dropout,
                     lm.initial_h_:h}
        cost, h, _ = session.run([loss, lm.final_h_, train_op],feed_dict=feed_dict)

        #### END(YOUR CODE) ####
        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print("[batch {:d}]: seen {:d} words at {:.1f} wps, loss = {:.3f}".format(
                i, total_words, avg_wps, avg_cost))
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [66]:
def run_gan_epoch(lm, session, words_to_ids, ids_to_words, train_list, batch_size, verbose=False, tick_s=10, learning_rate=None):
    assert(learning_rate is not None)
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0
    train_op = lm.train_step_
    loss = lm.loss_cnn

    #if train:
        #train_op = lm.train_step_
        #use_dropout = True
        ##loss = lm.train_loss_
        #loss = lm.loss_cnn
    #else:
        #train_op = tf.no_op()
        #use_dropout = False  # no dropout at test time
        #loss = lm.loss_  # true loss, if train_loss is an approximation
        #loss = lm.loss_cnn

    ### UPDATED!!! MUST PASS IN TRAIN_IDS as a list of lists, batch_size
    num_samples = 2*batch_size
    total_reviews = len(train_list)
    num_batches_per_epoch = int((total_reviews-1)/batch_size) + 1
    for batch in range(num_batches_per_epoch):
        print("gan batch: ", batch)
        start_index = batch*batch_size
        end_index = min((batch+1) * batch_size, total_reviews)
        current_training_batch = train_list[start_index:end_index]
        #min_review_length = min(len(review) for review in current_training_batch)
        min_review_length = 300 #based on the selection criteria when extracting data.  limits learning to ~60 word context
        #average_review_length = sum([len(review) for review in current_training_batch])/len(current_training_batch)
        #max_steps = 2.0*average_review_length
        max_steps = 325
        
        #for training_review in current_training_batch:
        w = np.repeat([[words_to_ids.get('<SOR>')]], num_samples, axis=0)#MUST PASS IN WORDS_TO_IDS
        h = session.run(lm.initial_h_, {lm.input_w_: w})
        #print(h[0])
        #print(h[0][0].shape)
        #print(h[0][1].shape)
        #can gather all outputs from this loop if you get to it
        #get the cell state and timestep 300
        h_artificial_300 = None
        for i in range(max_steps):
            if i == min_review_length:
                h_artificial_300 = h[0][1]
            h, y = sample_step(lm, session, w[:,-1:], h)
            w = np.hstack((w,y))
        artificial_review_final_states = []
        artificial_review_ids = []
        #for row in w:
        for a, row in enumerate(h_artificial_300):
            #print("generated during training", end=":  ")
            new_artificial_review = []
            for b, word_id in enumerate(w[a]):
                new_artificial_review.append(word_id)
                #print(ids_to_words[word_id], end="")
                if (b != 0) and (word_id == words_to_ids.get("<EOR>")):
                    break
            #print("")
            #if len(new_artificial_review) >= 0.75*average_review_length:
            if len(new_artificial_review) >= min_review_length:
                artificial_review_ids.append(new_artificial_review)
                artificial_review_final_states.append(row)
        
        print("generated during training batch ", batch, ":")
        for review in artificial_review_ids[:5]:
            for word_id in review:
                print(ids_to_words[word_id], end="")
            print()
        #print(artificial_review_ids[:5])
        #now you have current_training_batch and artificial_review_ids
        #clip all data to the same length for simplicity
        num_reviews = min(len(current_training_batch), len(artificial_review_ids))
        print("gan batch: ", batch, " has ", num_reviews, 
              " real reviews and ", num_reviews, " artificial reviews.")
        
        if num_reviews == 0:
            continue
        
        current_training_batch = [review[:min_review_length] for review in current_training_batch]
        #not sure this is needed
        artificial_review_ids = [review[:min_review_length] for review in artificial_review_ids]
        
        ##get final states for real reviews
        #w_training = np.array(current_training_batch)
        #h = session.run(lm.initial_h_, {lm.input_w_: w_training})
        #feed_dict = {lm.input_w_:w_training,
                     #lm.learning_rate_: learning_rate,
                     #lm.use_dropout_: False,
                     #lm.initial_h_:h}
        #h_real = session.run([lm.final_h_],feed_dict=feed_dict)#no training, just get states
        
        #real_review_final_states = []
        #prob don't need to do anything to h_real
        #for row in h_real:
            #if len(new_artificial_review) >= 300:
                #artificial_review_ids.append(new_artificial_review)
                #artificial_review_final_states.append(row)
                
        #now let's even out the number of examples
        current_training_batch = current_training_batch[:num_reviews]
        artificial_review_ids = artificial_review_ids[:num_reviews]
        #print(h_real[0])
        ##print(h_real[1])#wtf why won't this work
        ##print(h_real[1].shape)
        ##h_real = h_real[1][:num_reviews]
        #h_real = h_real[:num_reviews]
        #print(h_real)
        #artificial_review_final_states[:num_reviews]
        real_review_list_for_retraining_softmax = current_training_batch[:]
        
        #label each review and shuffle data
        current_training_batch = [(review,[1,0]) for review in current_training_batch]#might need to swap labels
        artificial_review_training_batch = [(np.array(review),[0,1]) for review in artificial_review_ids]
        #print(current_training_batch[:10])
        #print()
        #print(artificial_review_ids[:10])
        #print()
        ##label each review hidden state and shuffle
        #training_states = [(review,np.array([0,1])) for review in h_real]#might need to swap labels
        #artificial_review_states = [(review,np.array([1,0])) for review in artificial_review_final_states]
        
        #combine training lists
        current_training_batch.extend(artificial_review_training_batch)
        np.random.shuffle(current_training_batch)
        
        #print(current_training_batch[:10])
        #print()
        #np.random.shuffle(training_states)
        #print(training_states)
        
        #train discriminator
        #now unzip into "w" and "y"
        review_list, labels = zip(*current_training_batch)
        #review_states, labels = zip(*training_states)
        #convert to matrix form
        #print(labels)
        w = np.array(list(review_list))
        #w = np.array(list(review_states))
        #y = np.array(list(labels))
        y = np.array(labels)
        #print(w.shape)
        #print(y.shape)
        #print()
        #batching is done at the review level
        #the whole review is fed in at once, so initialize h first
        #if batch == 0:
        
        #train CNN classifier
        train_op = lm.train_step_
        h = session.run(lm.initial_h_, {lm.input_w_: w})
        #feed_dict = {lm.input_x:w,
                     #lm.input_y:y,
                     #lm.learning_rate_: learning_rate,
                     #lm.use_dropout_: True}
        feed_dict = {lm.input_w_: w, 
                     lm.input_y: y,
                     lm.learning_rate_: learning_rate,
                     lm.use_dropout_: True, 
                     lm.initial_h_: h}
        accuracy, cost, h, _ = session.run([lm.accuracy, loss, lm.final_h_, train_op],feed_dict=feed_dict)
        print("discriminator training accuracy for gan batch ", batch, " is: ", accuracy)
        print("discriminator training cost for gan batch ", batch, " is: ", cost)
        
        ##train discriminator on fake reviews
        ##now unzip into "w" and "y"
        #review_list, labels = zip(*artificial_review_ids)
        #w = np.array(list(review_list))
        #y = np.array(labels)
        
        ##train CNN classifier
        #train_op = self.train_step_
        #h = session.run(lm.initial_h_, {lm.input_w_: w})
        #feed_dict = {lm.input_w_: w, 
                     #lm.input_y: y,
                     #lm.learning_rate_: learning_rate,
                     #lm.use_dropout_: True, 
                     #lm.initial_h_: h}
        #accuracy, cost, h, _ = session.run([lm.accuracy, loss, lm.final_h_, train_op],feed_dict=feed_dict)
        #print("fake review accuracy for gan batch ", batch, " is: ", accuracy)
        #print("fake review cost for gan batch ", batch, " is: ", cost)
        
        #train generator
        #relabel fake reviews as real
        artificial_review_training_batch = [(np.array(review),[1,0]) for review in artificial_review_ids]
        #now unzip into "w" and "y"
        review_list, labels = zip(*artificial_review_training_batch)
        w = np.array(list(review_list))
        y = np.array(labels)
        
        #train CNN classifier
        train_op = lm.train_step1_
        h = session.run(lm.initial_h_, {lm.input_w_: w})
        #feed_dict = {lm.input_x:w,
                     #lm.input_y:y,
                     #lm.learning_rate_: learning_rate,
                     #lm.use_dropout_: True}
        feed_dict = {lm.input_w_: w, 
                     lm.input_y: y,
                     lm.learning_rate_: learning_rate,
                     lm.use_dropout_: True, 
                     lm.initial_h_: h}
        accuracy, cost, h, _ = session.run([lm.accuracy, loss, lm.final_h_, train_op],feed_dict=feed_dict)
        print("generator training accuracy for gan batch ", batch, " is: ", accuracy)
        print("generator training cost for gan batch ", batch, " is: ", cost)
        
        ##retrain softmax of rnn on real reviews
        #flattened_real_ids = np.array([item for sublist in real_review_list_for_retraining_softmax for item in sublist])
        ##print(flattened_real_ids[:10])
        ##print(flattened_real_ids.shape)
        ##print()
        #bi = utils.rnnlm_batch_generator(flattened_real_ids, batch_size, max_time=150)
        #for i, (w, y) in enumerate(bi):
            #cost = 0.0
            #if i == 0:
                #h = session.run(lm.initial_h_, {lm.input_w_: w})
            #feed_dict = {lm.input_w_: w, 
                         #lm.target_y_: y, 
                         #lm.learning_rate_: learning_rate,
                         #lm.use_dropout_: False,
                         #lm.initial_h_:h}
            #cost, h, _ = session.run([lm.train_loss_, lm.final_h_, lm.train_step_softmax_],feed_dict=feed_dict)
            #print("softmax re-training cost for gan batch ", batch, " is: ", cost)
    
    ### UPDATED!!!
    total_cost += cost#update
    total_batches = batch + 1
    total_words += w.size  # w.size = batch_size * max_time

    ##
    # Print average loss-so-far for epoch
    # If using train_loss_, this may be an underestimate.
    if verbose and (time.time() - tick_time >= tick_s):
        avg_cost = total_cost / total_batches
        avg_wps = total_words / (time.time() - start_time)
        print("[batch {:d}]: seen {:d} words at {:.1f} wps, loss = {:.3f}".format(
            batch, total_words, avg_wps, avg_cost))
        tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [71]:
def score_dataset(lm, session, ids, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils.rnnlm_batch_generator(ids, batch_size=100, max_time=100)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=0.0, train=False, 
                     verbose=False, tick_s=3600)
    print("{:s}: avg. loss: {:.03f}  (perplexity: {:.02f})".format(name, cost, np.exp(cost)))
    return cost


#build a list of list of characters from the 5-star reviews
def preprocess_review_series(review_series):
    review_list = []
    for new_review in review_series:
        clipped_review = new_review[2:-1]
        char_list = list(clipped_review.lower())
        semifinal_review = []
        last_char = ''
        for ascii_char in char_list:
            if ascii_char == '\\' or last_char == '\\':
                pass
            else:
                semifinal_review.append(ascii_char)
            last_char = ascii_char
        if len(semifinal_review) > 300:
            final_review = ['<SOR>'] + semifinal_review + ['<EOR>']
            #print(final_review)
            review_list.append(final_review)
    return review_list

def get_review_series(review_path = '/home/kalvin_kao/yelp_challenge_dataset/review.csv'):
    #review_path = '/home/kalvin_kao/yelp_challenge_dataset/review.csv'
    review_df = pd.read_csv(review_path)
    five_star_review_df = review_df[review_df['stars']==5]
    #five_star_review_series = five_star_review_df['text']
    return five_star_review_df['text']

def get_business_list(business_path = '/home/kalvin_kao/yelp_challenge_dataset/business.csv'):
    #business_path = '/home/kalvin_kao/yelp_challenge_dataset/business.csv'
    return pd.read_csv(business_path)

def split_train_test(review_list, training_samples, test_samples):
    #pass in randomized review list
    train_len = int(np.floor(0.8*len(review_list)))
    test_len = int(np.floor(0.2*len(review_list)))
    training_review_list = review_list[:train_len]
    testing_review_list = review_list[-test_len:]
    randomized_training_list = random.sample(training_review_list, training_samples)
    randomized_testing_list = random.sample(testing_review_list, test_samples)
    training_review_list = [item for sublist in randomized_training_list for item in sublist]
    print("number of training characters", len(training_review_list))
    test_review_list = [item for sublist in randomized_testing_list for item in sublist]
    print("number of test characters", len(test_review_list))
    return randomized_training_list, randomized_testing_list

def make_train_test_data(five_star_review_series, training_samples=20000, test_samples=1000):
    #fix randomization to prevent evaluation on trained samples
    review_list = preprocess_review_series(five_star_review_series)
    #split and shuffle the data
    train_len = int(np.floor(0.8*len(review_list)))
    test_len = int(np.floor(0.2*len(review_list)))
    np.random.shuffle(review_list)
    training_review_list = review_list[:train_len]
    testing_review_list = review_list[-test_len:]
    randomized_training_list = random.sample(training_review_list, training_samples)
    randomized_testing_list = random.sample(testing_review_list, test_samples)
    #training_review_list = [item for sublist in review_list[:training_samples] for item in sublist]
    training_review_list = [item for sublist in randomized_training_list for item in sublist]
    print("number of training characters", len(training_review_list))
    
    #test_review_list = [item for sublist in review_list[training_samples:training_samples+test_samples] for item in sublist]
    test_review_list = [item for sublist in randomized_testing_list for item in sublist]
    print("number of test characters", len(test_review_list))
    return training_review_list, test_review_list


#def make_vocabulary(training_review_list, test_review_list):
#    unique_characters = list(set(training_review_list + test_review_list))
#    #vocabulary
#    char_dict = {w:i for i, w in enumerate(unique_characters)}
#    ids_to_words = {v: k for k, v in char_dict.items()}
#    return char_dict, ids_to_words
def make_vocabulary(dataset_list):
    unique_characters = list(set().union(*dataset_list))
    #unique_characters = list(set(training_review_list + test_review_list))
    #vocabulary
    char_dict = {w:i for i, w in enumerate(unique_characters)}
    ids_to_words = {v: k for k, v in char_dict.items()}
    return char_dict, ids_to_words

def convert_to_ids(char_dict, review_list):
    #convert to flat (1D) np.array(int) of ids
    review_ids = [char_dict.get(token) for token in review_list]
    return np.array(review_ids)

def run_training(train_list, test_ids, words_to_ids, ids_to_words, tf_savedir, model_params, max_time=100, batch_size=256, learning_rate=0.002, num_epochs=20):
    #V = len(words_to_ids.keys())
    # Training parameters
    ## add parameter sets for each attack/defense configuration
    #max_time = 25
    #batch_size = 100
    #learning_rate = 0.01
    #num_epochs = 10
    
    # Model parameters
    #model_params = dict(V=vocab.size, 
                        #H=200, 
                        #softmax_ns=200,
                        #num_layers=2)
    #model_params = dict(V=len(words_to_ids.keys()), 
                        #H=1024, 
                        #softmax_ns=len(words_to_ids.keys()),
                        #num_layers=2)
    #model_params = dict(V=V, H=H, softmax_ns=softmax_ns, num_layers=num_layers)
    
    #TF_SAVEDIR = "/tmp/artificial_hotel_reviews/a4_model"
    TF_SAVEDIR = tf_savedir
    checkpoint_filename = os.path.join(TF_SAVEDIR, "gan")
    trained_filename = os.path.join(TF_SAVEDIR, "gan_trained")
    
    # Will print status every this many seconds
    #print_interval = 5
    print_interval = 30
    
    lm = rnnlm.RNNLM(**model_params)
    lm.BuildCoreGraph()
    lm.BuildTrainGraph()
    ### UPDATED!!!
    lm.BuildSamplerGraph()
    #num_pretrain = 2000
    num_pretrain = 50#low number for testing
    #pretrain on a different set of training data each time
    #train and test ids 
    #pre_train_ids =
    #train_ids = 
    ### UPDATED!!!
    
    # Explicitly add global initializer and variable saver to LM graph
    with lm.graph.as_default():
        initializer = tf.global_variables_initializer()
        saver = tf.train.Saver()
        
    # Clear old log directory
    shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
    if not os.path.isdir(TF_SAVEDIR):
        os.makedirs(TF_SAVEDIR)
    
    with tf.Session(graph=lm.graph) as session:
        # Seed RNG for repeatability
        #tf.set_random_seed(42)
    
        session.run(initializer)
        
        #check trainable variables
        #variables_names = [v.name for v in tf.trainable_variables()]
        #values = session.run(variables_names)
        #for k, v in zip(variables_names, values):
            #print("Variable: ", k)
            #print("Shape: ", v.shape)
            #print(v)
    
        for epoch in range(1,num_epochs+1):
            t0_epoch = time.time()
            np.random.shuffle(train_list)
            #shuffled_train_list = np.random.shuffle(train_list)
            #pre_train_ids = [item for sublist in shuffled_train_list[:num_pretrain] for item in sublist]
            #gan_train_list = shuffled_train_list[num_pretrain:]
            #train_list = 
            pre_train_ids = np.array([item for sublist in train_list[:num_pretrain] for item in sublist])
            #print(pre_train_ids.shape)
            #pre_train_ids = np.array(pre_train_ids)
            gan_train_list = train_list[num_pretrain:]
            if num_pretrain > 0:
                bi = utils.rnnlm_batch_generator(pre_train_ids, batch_size, max_time)
                print("[epoch {:d}] Starting epoch {:d}".format(epoch, epoch))
                # Run a pretraining epoch.
                run_epoch(lm, session, batch_iterator=bi, train=True, verbose=True, tick_s=10, learning_rate=learning_rate)

                print("[epoch {:d}] Completed in {:s}".format(epoch, utils.pretty_timedelta(since=t0_epoch)))
        
            #now train gan
            #run_gan_epoch(lm, session, words_to_ids, ids_to_words, train_list, verbose=False, tick_s=10, learning_rate=None)
            run_gan_epoch(lm, session, words_to_ids, ids_to_words, gan_train_list, batch_size, 
                          verbose=True, tick_s=10, learning_rate=learning_rate)
        
            # Save a checkpoint
            saver.save(session, checkpoint_filename, global_step=epoch)
        
            ##
            # score_dataset will run a forward pass over the entire dataset
            # and report perplexity scores. This can be slow (around 1/2 to 
            # 1/4 as long as a full epoch), so you may want to comment it out
            # to speed up training on a slow machine. Be sure to run it at the 
            # end to evaluate your score.
            print("[epoch {:d}]".format(epoch), end=" ")
            score_dataset(lm, session, pre_train_ids, name="Train set")
            print("[epoch {:d}]".format(epoch), end=" ")
            score_dataset(lm, session, test_ids, name="Test set")
            print("")
        
        # Save final model
        saver.save(session, trained_filename)
        return trained_filename

def get_char_probs(trained_filename, model_params, test_ids):
    lm = rnnlm.RNNLM(**model_params)
    lm.BuildCoreGraph()
    all_review_likelihoods = []
    train_op = tf.no_op()
    use_dropout = False
    loss = lm.loss_
    
    with lm.graph.as_default():
        saver = tf.train.Saver()
    
    with tf.Session(graph=lm.graph) as session:
        #train_op = tf.no_op()
        #use_dropout = False
        #loss = lm.loss_
        
        saver.restore(session, trained_filename)
        
        for review in test_ids:
            review_likelihoods = []
            inputs = review[:-1]
            labels = review[1:]
            inputs_labels = zip(inputs,labels)
            for i, (w,y) in enumerate(inputs_labels):
                
                w = np.array(w)
                y = np.array(y)
                w = w.reshape([1,1])
                y = y.reshape([1,1])
                
                if i == 0:
                    h = session.run(lm.initial_h_, {lm.input_w_: w})

                feed_dict = {lm.input_w_:w, 
                             lm.target_y_:y,
                             lm.learning_rate_: 0.002,
                             lm.use_dropout_: use_dropout,
                             lm.initial_h_:h}
                cost, h = session.run([loss, lm.final_h_],feed_dict=feed_dict)
                likelihood = 2**(-1*cost)
                review_likelihoods.append(likelihood)
            all_review_likelihoods.append(review_likelihoods)
    return all_review_likelihoods

## Sampling
def sample_step(lm, session, input_w, initial_h):
    """Run a single RNN step and return sampled predictions.
  
    Args:
      lm : rnnlm.RNNLM
      session: tf.Session
      input_w : [batch_size] vector of indices
      initial_h : [batch_size, hidden_dims] initial state
    
    Returns:
      final_h : final hidden state, compatible with initial_h
      samples : [batch_size, 1] vector of indices
    """
    # Reshape input to column vector
    input_w = np.array(input_w, dtype=np.int32).reshape([-1,1])

    # Run sample ops
    feed_dict = {lm.input_w_:input_w, lm.initial_h_:initial_h}
    final_h, samples = session.run([lm.final_h_, lm.pred_samples_], feed_dict=feed_dict)

    # Note indexing here: 
    #   [batch_size, max_time, 1] -> [batch_size, 1]
    return final_h, samples[:,-1,:]

def generate_text(trained_filename, model_params, words_to_ids, ids_to_words):
    # Same as above, but as a batch
    #max_steps = 20
    max_steps = 300
    #num_samples = 40000
    num_samples = 40
    random_seed = 42
    
    lm = rnnlm.RNNLM(**model_params)
    lm.BuildCoreGraph()
    lm.BuildSamplerGraph()
    
    with lm.graph.as_default():
        saver = tf.train.Saver()
    
    with tf.Session(graph=lm.graph) as session:
        # Seed RNG for repeatability
        #tf.set_random_seed(random_seed)
        
        # Load the trained model
        saver.restore(session, trained_filename)
    
        # Make initial state for a batch with batch_size = num_samples
        #w = np.repeat([[vocab.START_ID]], num_samples, axis=0)
        w = np.repeat([[words_to_ids.get('<SOR>')]], num_samples, axis=0)
        h = session.run(lm.initial_h_, {lm.input_w_: w})
        # take one step for each sequence on each iteration 
        for i in range(max_steps):
            h, y = sample_step(lm, session, w[:,-1:], h)
            w = np.hstack((w,y))
    
        # Print generated sentences
        for row in w:
            print(trained_filename, end=":  ")
            for i, word_id in enumerate(row):
                #print(vocab.id_to_word[word_id], end=" ")
                print(ids_to_words[word_id], end="")
                #if (i != 0) and (word_id == vocab.START_ID):
                if (i != 0) and (word_id == words_to_ids.get("<EOR>")):
                    break
            print("")

def train_attack_model(training_samples=20000, test_samples=1000, review_path = '/home/kalvin_kao/yelp_challenge_dataset/review.csv'):
    #training_samples=20000
    #test_samples=1000
    #review_path = '/home/kalvin_kao/yelp_challenge_dataset/review.csv'
    start_format = time.time()
    five_star_reviews = get_review_series(review_path)
    train_review_list, test_review_list = make_train_test_data(five_star_reviews, training_samples, test_samples)
    words_to_ids, ids_to_words = make_vocabulary(train_review_list, test_review_list)
    train_ids = convert_to_ids(words_to_ids, train_review_list)
    test_ids = convert_to_ids(words_to_ids, test_review_list)
    end_format = time.time()
    print("data formatting took " + str(end_format-start_format) + " seconds")
    model_params = dict(V=len(words_to_ids.keys()), 
                            H=1024, 
                            softmax_ns=len(words_to_ids.keys()),
                            num_layers=2)
    #run_training(train_ids, test_ids, tf_savedir, model_params, max_time=100, batch_size=256, learning_rate=0.002, num_epochs=20)
    trained_filename = run_training(train_ids, test_ids, tf_savedir = "/tmp/artificial_hotel_reviews/a4_model", model_params=model_params, max_time=150, batch_size=256, learning_rate=0.002, num_epochs=20)
    return trained_filename, model_params, words_to_ids, ids_to_words

def neg_log_lik_ratio(likelihoods_real, likelihoods_artificial):
    predictions = []
    combined = zip(likelihoods_real, likelihoods_artificial)
    for (real_review_likelihoods, artificial_review_likelihoods) in combined:
        negative_log_lik_ratios = -1*(np.log(np.divide(real_review_likelihoods, artificial_review_likelihoods)))
        #averaged_llrs = negative_log_lik_ratios[:-1]/(len(negative_log_lik_ratios)-1)
        averaged_llrs = np.sum(negative_log_lik_ratios[:-1])/(len(negative_log_lik_ratios)-1)
        predictions.append(averaged_llrs)
    return predictions

In [5]:
start_dl = time.time()
os.system('gsutil -q cp gs://w266_final_project_kk/data/split01_train_data_03.csv .')
os.system('gsutil -q cp gs://w266_final_project_kk/data/split01_test_data_03.csv .')
#os.system('gsutil -q cp gs://w266_final_project_kk/data/gen01_train_data_01.csv .')
#os.system('gsutil -q cp gs://w266_final_project_kk/data/gen01_test_data_01.csv .')
end_dl = time.time()
print("data download took " + str(end_dl-start_dl) + " seconds")
#gsutil cp gs://[BUCKET_NAME]/[OBJECT_NAME] [OBJECT_DESTINATION]
real_train_review_path = './split01_train_data_02.csv'
real_test_review_path = './split01_test_data_02.csv'
#artificial_train_review_path = './gen01_train_data_01.csv'
#artificial_test_review_path = './gen01_test_data_01.csv'

data download took 4.435562372207642 seconds


In [6]:
real_train_review_path = '/home/kalvin_kao/final_project/split01_train_data_02.csv'
real_test_review_path = '/home/kalvin_kao/final_project/split01_test_data_02.csv'

In [7]:
start_open = time.time()
with open(real_train_review_path, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    training_review_list_real = [sublist for sublist in reader]
training_review_list_real_training_eval = [item for sublist in training_review_list_real for item in sublist]

with open(real_test_review_path, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    test_review_list_real = [sublist for sublist in reader]
    #make into list of list
test_review_list_real_training_eval = [item for sublist in test_review_list_real for item in sublist]

#with open(artificial_train_review_path, 'r') as csvfile:
    #reader = csv.reader(csvfile, delimiter=',')
    #training_review_list_artificial = [item for sublist in reader for item in sublist]

#with open(artificial_test_review_path, 'r') as csvfile:
    #reader = csv.reader(csvfile, delimiter=',')
    #test_review_list_artificial = [sublist for sublist in reader]
    #make into list of list
#test_review_list_artificial_training_eval = [item for sublist in test_review_list_artificial for item in sublist]
end_open = time.time()
print("data reading took " + str(end_open-start_open) + " seconds")

data reading took 2.0584964752197266 seconds


In [8]:
start_vocab = time.time()
#words_to_ids, ids_to_words = make_vocabulary([training_review_list_real, test_review_list_real_training_eval, training_review_list_artificial, test_review_list_artificial_training_eval])
words_to_ids, ids_to_words = make_vocabulary([training_review_list_real_training_eval, test_review_list_real_training_eval])
train_ids_real = [convert_to_ids(words_to_ids, review) for review in training_review_list_real]
train_ids_real_training_eval = convert_to_ids(words_to_ids, training_review_list_real_training_eval)
test_ids_real = [convert_to_ids(words_to_ids, review) for review in test_review_list_real]
test_ids_real_training_eval = convert_to_ids(words_to_ids, test_review_list_real_training_eval)
#train_ids_artificial = convert_to_ids(words_to_ids, training_review_list_artificial)
#test_ids_artificial = [convert_to_ids(words_to_ids, review) for review in test_review_list_artificial]
#test_ids_artificial_training_eval = convert_to_ids(words_to_ids, test_review_list_artificial_training_eval)
end_vocab = time.time()
print("vocabulary building took " + str(end_vocab-start_vocab) + " seconds")

vocabulary building took 5.9633660316467285 seconds


In [9]:
print(np.random.shuffle(train_ids_real[:10]))

None


In [72]:
reload(rnnlm)

<module 'trainer.rnnlm' from '/home/kalvin_kao/artificial_hotel_reviews/model_dev/gan/trainer/rnnlm.py'>

In [ ]:
start_training = time.time()
#model_params = dict(V=len(words_to_ids.keys()), H=1024, softmax_ns=len(words_to_ids.keys()), num_layers=2)
model_params = dict(V=len(words_to_ids.keys()), H=600, softmax_ns=len(words_to_ids.keys()), num_layers=1)#low numbers for dev
#trained_filename_real = run_training(train_ids_real, test_ids_real_training_eval, tf_savedir = "/tmp/defense_model/real", model_params=model_params, max_time=150, batch_size=128, learning_rate=0.002, num_epochs=20)
#trained_filename_artificial = run_training(train_ids_artificial, test_ids_artificial_training_eval, tf_savedir = "/tmp/defense_model/artificial", model_params=model_params, max_time=150, batch_size=128, learning_rate=0.002, num_epochs=20)
#run_training(train_ids, test_ids, words_to_ids, ids_to_words, tf_savedir, model_params, max_time=100, batch_size=256, learning_rate=0.002, num_epochs=20)
trained_filename = run_training(train_ids_real[:10000], test_ids_real_training_eval[:1000000], words_to_ids, ids_to_words, tf_savedir = "/tmp/gan_model/practice", model_params=model_params, max_time=150, batch_size=96, learning_rate=0.01, num_epochs=1)#UPDATE FOR ACTUAL RUN
#trained_filename_artificial = run_training(train_ids_artificial, test_ids_artificial_training_eval[:1000000], tf_savedir = "/tmp/defense_model/artificial", model_params=model_params, max_time=150, batch_size=128, learning_rate=0.002, num_epochs=20)#UPDATE FOR ACTUAL RUN
end_training = time.time()
print("overall training took " + str(end_training-start_training) + " seconds")

[epoch 1] Starting epoch 1
[epoch 1] Completed in 0:00:04
gan batch:  0
generated during training batch  0 :
<SOR>;#q meristhersthererererererederererersthisthererstherestherererereresthestherereristhisthereristhisthistherererererererererereristherereristhererstheristherersthistherersthereristhistherlly histherererererstherersustheresthererstherersthisthererstheristheresthereristhistheristheristherastherersthisthistheredersthesthererer
<SOR>herererstheristhisthisthistheristheresthisthererersthereristhistheristheristhereristhererestheristhisthisthererstherererererererererererestherererererereristhereredesthererererererstheristhistheristheristhisthersthisthererererererousthereristhererstheristhererestherererstherstheristhisthisthereristhisthistheristhererngedere
<SOR>qchererererererereristhisthesthererersthistheristheristhererstheroustheristhererestheristherererersthisthereristheristhererererersthisthererereristherereristhelly merererererererererererousthisthesthererersthistheresthereris

discriminator training accuracy for gan batch  4  is:  0.5
discriminator training cost for gan batch  4  is:  22.0293
generator training accuracy for gan batch  4  is:  1.0
generator training cost for gan batch  4  is:  0.0143406
gan batch:  5
generated during training batch  5 :
<SOR>ingthesisthelllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll
<SOR>q? hesthesthesthesthallllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll
<SOR>q? whersthesthesthesthessthesthesthesthesthesthesthe

generated during training batch  9 :
<SOR>q hestheluuerstherstouthersthelulueluduluuerstheluluelulutestheluueluthestheluuellurelutersthesthoustheluueluuelutherstouthestheluuestelutherstoutheluluterstoustheluueluuerrstheluuereluthelutherstheluluouelutherstheluluterstheluteluluuersthellududutouthelutesthestheluueluthersthesthesthelulutheluoustherthstheluthesthersthel
<SOR>~ muelutouthesthesthestheluuersthestheluthelulueluluestheluluthesthesthesthelustheluuelutheluthesthestheluteluuersthersthelutereluthesthsthethelluuerelluduuelludueluluuestheluterstheluluthesthestheluoueluueludulutoutherstelutersthestheluuelunthelutheluduthaluluuelurelludustheluueluluterstestheluutoutheludutheluueresthersthe
<SOR>8 muterlutersthelutestheluthesthsthestheluuerstheluduistheluduluuelutheluluthesthelutoustheluthesthesthesthesthersthelutherersteluluthersthereluduluthestheluueluuelutheluthelubelulu-elutheluluthelutheluduluresthelluueresthelutheluthesthestherstheluuerersthelutoueristhelulutoustheluuelluduluthest

discriminator training accuracy for gan batch  13  is:  0.5
discriminator training cost for gan batch  13  is:  0.691112
generator training accuracy for gan batch  13  is:  0.5625
generator training cost for gan batch  13  is:  1.02576
gan batch:  14
generated during training batch  14 :
<SOR>8 hesrrterelueluerelurelueresterestoueeluerterestouerelloueresterestoustertouerestesterellueresterelueluestererelluelureluelueluerelueresteresestouesteluerelluelurelueluererelluieluluresterestedrelluerelluresterestouestererereluereluerestesterereluesterelurelueresrtertestesterelurestelluereluelueeluereluerestesteretestouest
<SOR>q hesterelueererestedrterestestestouerelueluerelueluerelueerestouereluellueelluesterterelluerellueresteresterererestestellueresterelueluereluaisterelluereluerellurellurerellueserestelutererelluelurelulurestouerelueellueresteresterellueresterelueluresreresteluerestouerellluerellurelueluerelluerelueluseesreluerelueelueerelulu
<SOR>q hererelullueelluertestoueelueesteluereluee

generated during training batch  18 :
<SOR>~ wuesttthtthtthtthththotthithetthttothstthotettothototothesthettthetththothoththothothustthththothothothotthothstthothettotthetthitisthethotthotthetttthotththothothotothotthththothototthoththothottthetetthothisthisthetthothothothothothotththotttithetettetthotthotthothothithethotthtthettthetttthothottistisththttottthotthett
<SOR>q hesthettthtoththettetththothothothotthotothoouthetetttthotthttthotothtisththtottotthottotisthothetthotottthotthothothusthotthothothotttthetettthotothothtothththothoththothothetthothothetthttotttittithothstthesthetthothoththstthtthotothothotttthothototothotttthetthethetthettotthotthothotothotthottthottotthsthoththothothoot
<SOR>q ouesttttittthetthottothothetthotothotthothothetthettththttthothothothottistthotthotthotothothotththottotthotouusthesthettthothettthotothetthotthtithetthothothothtthoththotthisthistithothotththeththoustthottothothothothetthotthetthothotthotthothisthotothestththtithotththettthisthothothotisttot

...

generated during training batch  63 :
<SOR>q istththsitttsssthsisththtttsthithothtttttsththottthottsttttttttttthottttthtststthottstttttthttthittththsttthsitttthothsstthtttttsttstththtstttstthstthttsttthsthslotttthttthottthittthttstttttttttttthithtttttthttttttttttttttthttttsttthstholotttttttttttttsittttthitttthittttsttttttthstttttttsitthtstttsththttthstttththththsttt
<SOR>q isthsttssttthistsststthstttthottsthsthsthotsrtthitttttutttthstthstthtthtttthothstttssttthtthottitthotththsittthttsthttstthithsthsittthsssthtthistthstthstthssthtsrtsthotttttthortttthtstttttttthststthottttttttsthsttththssststthottthssttththottittsittthstttthtsththottstthottttstttststtthstttthttsttttttsthsththotttthsthsssttt
<SOR>~ hettthottttthsiiststhtsthitthttshsthttthottslothtttttttttttthtittttttthistthtttttthttstttttttthithttitstthtttstttthttittttttthothtththstthttithtttttttthtttththtottttttthsittttttttsthotttthtttttttttthttthtstttthisitthtttthsttthtssthothtithotiststhtsttittstttittthtttthottthlottttttttthtttttttst

discriminator training accuracy for gan batch  67  is:  0.5
discriminator training cost for gan batch  67  is:  0.693148
generator training accuracy for gan batch  67  is:  0.0
generator training cost for gan batch  67  is:  0.693938
gan batch:  68
generated during training batch  68 :
<SOR>q htittthotththiththththertththotsttttttttthstttttthsithttttthttttthtththssthittthtthotsthtttsthttththotttthtttttttttsttsstttthothotthtthtththsitttttthtstthtthtththoottsttttthtttttttthtttsiththsttttthtstthththithttslotitttthttttttthtttttttttttttttttttttthitttitttttththththtttttttttttsttttttttthstttthothtstttttthtthtithsttth
<SOR>q istththtthoustthttsttttthtthtttthttisttthssstttsstttttthistthsthistthtthisthssttsithothsrttssthttstththstttstttthtthththsisttthttthsthsttththsttthssttthshstththstthottthstsithoustththththsthsthothsthistthottstthttththttthstttthttslothttttttttttthtttthotttthtttttttthtttttthtttttttthittttttthtthtthttthtsthttttssithottthitth
<SOR>q ithtttsthrttststttthtthoustththttthstthithst

generated during training batch  72 :
<SOR>q ithoutthotstthittttthttttttthttthtthsttttttthttttsthtthithsthisttsisthotthsttthsthsttsttthttthistthtttthithtsthththsitttthitthsthtrusthitthtthtsttttthsttsthoutthtthotttttthsthttthottthtttsttsthttttsthoutthstttsthstttsthssttstthttsthstthtttstthertttthottththoutththotttthstttttttthsthtttttttttsttthtttsthtththolotthithstttth
<SOR>q istttthtthsrttthstttththsttitthssthssttthttststthtthsssttthtssthttstthsttthsthsisthsttththsstttthstttththsthotsttttsthssststthothothtthsttthtisthsthisttthoththittslottttttthittttttttthtthithitttthtttthtththsttttssttttsistththssttttthtsttittstththsthssthsthitttthstttotthsthththtthstthittttthttthsththitttsthistthotthssithtt
<SOR>q hertlottttttthsttthtthsttttttthttttttttttttttttittttttttttttttthtthsttthsstthsttththsthsttttstthttsttthsthtthtststtthsthstthothssthothsistththtthothththtthouutttttthtttttthtttthttstthstttssthttttttstttthththtsttsitstthithothsthtthitthsttttsttthtsttthotththithtsttthstthottttttthtthstttstthitht

discriminator training accuracy for gan batch  76  is:  0.5
discriminator training cost for gan batch  76  is:  0.693147
generator training accuracy for gan batch  76  is:  1.0
generator training cost for gan batch  76  is:  0.692687
gan batch:  77
generated during training batch  77 :
<SOR>q ittststssththsttthotthttttttthsthtistthsthtsiistthtsiithtthitsthlotttttttthtttththotthotttttthtthtttittstthttthottthtttthttthtttttttstttttitttttttttthttttttttttttttthstttitttthsttththttsthtthttsitthotttththtthsstththotthttttthtthttthtthtttthtthittttittthttthtthttthstththsttttthottthtttthtttttttththottittttttthithtthsttttt
<SOR>q isthststtthisttthstholotttttttttthtthstttttttttttttthttthstthottttthtttttthsttttthittttttththttthtttttttttttssththsthththtstttttttttthsthttththtthttstththsttttthttthstttttthittththsttthitsttthttithsthstthtthtstttstttthisthttthtstthouttthottttstthotttttthottsstttttstthstthttsststttttthsththithslotthttttttttthtttthttttttttt
<SOR>~ otsttththttttttsttthtthsththtthttttthithoutt

generated during training batch  81 :
<SOR>q ithtsthsitthtthstsththstthsthtthtstthtttttstththtthssttthothsthsthotthitthittsttitthsitthittttttthithothtthtsthottthoutthtttttthisstttittttttttttththitthttthtittsithtthstthtttttttsthothttsttthstttsthittsththoustttthththsttthttthttstttsttthitsththisstttthsththotthstthttthotrlottrttttttttttthttittthtttthtttttttthsthttttttth
<SOR>q istttsthtststthotstthittthottsttsthtthtthistthistththsthtttttsthsthsthoththtthottthttsttthtttthttttthottttstthtstttttttittttstthtththotsttisttttstthsttthtthsthsthotsttstttttthsthstthsttttttsthitthittththottthistthttttstthsthsttthsthsttstthitthouustthistthsisthththtthssstthsthottthstthtthsithoutthisstttthtttthothststtthtth
<SOR>q hetththsitthsttttittttrttttststthoutththittthotttthstholotttttttittthitttttttttthitttsitttstttttttsttttthitlotttthtsthtssttttstttttttttthttttttttthstttthittttttttsttttttttttsttttttttththttttthststtttittttthttthisthithttstthttthttthsthttthssttthtttsitthttthiitthitttttthssttththsssthisithtthtts

In [ ]:
### UPDATED!!!
#save both RNNs for later use
save_command_1  = "gsutil cp -r " + trained_filename_real[0:trained_filename_real.rfind("/")] + " gs://w266_final_project_kk/defense_baseline/real/" + str(int(np.floor(time.time())))
save_command_2  = "gsutil cp -r " + trained_filename_artificial[0:trained_filename_artificial.rfind("/")] + " gs://w266_final_project_kk/defense_baseline/artificial/" + str(int(np.floor(time.time())))
#save_command_1  = "gsutil cp -r " + trained_filename_real[0:trained_filename_real.rfind("/")] + " gs://w266_final_project_kk/practice_run/defense_real/" + str(int(np.floor(time.time())))
#save_command_2  = "gsutil cp -r " + trained_filename_artificial[0:trained_filename_artificial.rfind("/")] + " gs://w266_final_project_kk/practice_run/defense_artificial/" + str(int(np.floor(time.time())))
os.system(save_command_1)
os.system(save_command_2)
### UPDATED!!!

#generate examples from each GAN out of curiosity
start_sampling = time.time()
generate_text(trained_filename, model_params, words_to_ids, ids_to_words)
#generate_text(trained_filename_artificial, model_params, words_to_ids, ids_to_words)
end_sampling = time.time()
print("character sampling took " + str(end_sampling-start_sampling) + " seconds")

#

#first feed the real reviews into each RNN and get the softmax probability of each character
#get the classification for real reviews by forming an average negative log-likelihood ratio for each review
start_scoring = time.time()
test_likelihoods_real_from_real = get_char_probs(trained_filename_real, model_params, test_ids_real[:1000])
test_likelihoods_real_from_artificial = get_char_probs(trained_filename_artificial, model_params, test_ids_real[:1000])
predictions_real = neg_log_lik_ratio(test_likelihoods_real_from_real, test_likelihoods_real_from_artificial)
#negative_log_lik_ratios = -1*(np.log(np.divide(test_likelihoods_real_from_real, test_likelihoods_real_from_artificial)))
#predictor = 

#next feed the generated reviews into each RNN and get the softmax probability of each character
#get the classification for generated reviews by forming an average negative log-likelihood ratio for each review
test_likelihoods_artificial_from_real = get_char_probs(trained_filename_real, model_params, test_ids_artificial[:1000])
test_likelihoods_artificial_from_artificial = get_char_probs(trained_filename_artificial, model_params, test_ids_artificial[:1000])
predictions_artificial = neg_log_lik_ratio(test_likelihoods_artificial_from_real, test_likelihoods_artificial_from_artificial)
end_scoring = time.time()
print("review scoring took " + str(end_scoring-start_scoring) + " seconds")

### UPDATED!!!
predictions_real = np.array(predictions_real)
predictions_artificial = np.array(predictions_artificial)
np.savetxt("predictions_real.csv", predictions_real, delimiter=",")
np.savetxt("predictions_artificial.csv", predictions_artificial, delimiter=",")
os.system("gsutil cp predictions_real.csv gs://w266_final_project_kk/defense_baseline/predictions_real/")
os.system("gsutil cp predictions_artificial.csv gs://w266_final_project_kk/defense_baseline/predictions_artificial/")
#os.system("gsutil cp predictions_real.csv gs://w266_final_project_kk/practice_run/defense_predictions_real/")
#os.system("gsutil cp predictions_artificial.csv gs://w266_final_project_kk/practice_run/defense_predictions_artificial/")
### UPDATED!!!